# Project Objective


Microsoft is creating a new movie studio. This project aims to advise Microsoft on what type of movies to produce. More specifically, it presents three actionable recommendations. 

# Recommendations

The three recommendations derived from our analysis are:
    
    1. Adopt the less riskier-larger budget business model (> $90 million)

    2. Exploiting the synergies between Microsoft’s existing assets/capabilities, focus on animation movies
 
    3. Release the movie in a month with large audience numbers but less competition - October


The following is a technical description of the process that is driving these recommendations. 

# Process

### Data Cleaning

We used two sources for our data: Two IMDB and two Movie DB files. IMDB provided basic movie characteristics and ratings while the Movie DB gave budget and revenues along with similarly basic movie characteristics.
We first merged the two files provided by the same source. The IMDB files contained a unique ID for each movie, which became key for the merge. The Movie DB files were merged with the release year-month and movie name used as the key. Movie names had to be processed to eliminate whitespaces and non-alphanumeric characters, and ware converted to uppercase.
Duplicate observations were also dropped.
As the last step, two sets of files were merged using the year and movie name as the joint key. 

In [ ]:
import Movie_Analysis_Functions # This file contains the functions created for this project.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data_directory = '/Users/flatironschol/fis_projects/Data/'  
df_imdb_basics = pd.read_csv(data_directory+'imdb.title.basics.csv.gz') # Source: IMDB
df_imdb_ratings = pd.read_csv(data_directory+'imdb.title.ratings.csv.gz') # Source: IMDB
df_mdb_ratings = pd.read_csv(data_directory+'tmdb.movies.csv.gz') # Source: MovieDB
df_mdb_revenues = pd.read_csv(data_directory+'tn.movie_budgets.csv.gz') # Source: MovieDB

In [ ]:
df_mdb_ratings.drop('Unnamed: 0', axis = 1, inplace = True) # This is an extra index column
df_mdb_ratings = df_clean(df_mdb_ratings, 'release_date', 'title')
df_mdb_revenues = df_clean(df_mdb_revenues, 'release_date', 'movie')
df_mdb = pd.merge(df_mdb_ratings, df_mdb_revenues, how = 'outer', \
              left_on = ['title', 'release_year_month', 'release_year'], \
              right_on = ['title', 'release_year_month', 'release_year'], \
              indicator = True, validate="one_to_one", \
              sort = True)

# Merge IMDB dataframes
df_imdb_basics = df_clean(df_imdb_basics, 'start_year', 'primary_title')
df_imdb = pd.merge(df_imdb_basics, df_imdb_ratings, how = 'outer', \
              on = 'tconst', \
              indicator = True, validate="one_to_one")

# Merge Movie DB and IMDB dataframes
df = pd.merge(df_imdb.drop(columns = '_merge'), df_mdb.drop(columns = '_merge'), how = 'outer', \
              left_on = ['title', 'release_year'], \
              right_on = ['title', 'release_year'], \
              indicator = True, validate="many_to_many", \
              sort = True)
df_merged = df.loc[df._merge == 'both']

# Clean the merged dataframe
df_clean = df.drop(columns = ['tconst', 'original_title_x', 'runtime_minutes', 'id_x', 'original_language', 'original_title_y', 'id_y'])
df_clean_drop = df_clean.loc[(df_clean.numvotes.isna()) & (df_clean.vote_count.isna())]
df_clean = df_clean.drop(df_clean_drop.index)

df_clean.to_csv (data_directory+'export_df_clean.csv', index = None, header=True) 

### Descriptive Analysis

The analysis focuses on two outcome indicators of interest: return on investment and audience numbers. For the first, we use log difference between the worldwide gross revenues and production budget to reduce dispersion. For the second, we use number of votes for the IMDB ratings. We prefer to use the IMDB ratings as the movie coverage is larger compared to the Movie DB ratings.
We link these outcome indicators to three movie characteristics: budget, release month, and genre. Budget refers to the production budget in log terms, similarly to reduce dispersion. The release month is extracted from release date (available only for the Movie DB data). The genre analysis does not classify each movie strictly under a genre. If a movie is classifed under multiple genres such as animation and comedy, the same movie will appear under both the list for animation movies and the list for comedy movies.
In exploring the links between budget and return on investment, outliers in the budget and revenue series are identified using the standard Q1-1.5IQR, Q3+1.5IQR formula. These outliers, for the time being, are dropped and will be subject to further analysis.  

In [3]:
data_directory = '/Users/flatironschol/fis_projects/Data/' 
import Movie_Analysis_Functions # This file contains the functions created for this project.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_clean = pd.read_csv(data_directory+'export_df_clean.csv')
df_budget = df_clean.loc[(df_clean.production_budget.isna() == False) & \
                         (df_clean.domestic_gross.isna() == False) & \
                         (df_clean.worldwide_gross.isna() == False)]
# Convert budget and revenues to int and log terms using 
# clean_dollar_values function from Movie_Analysis_Functions Python file.
for series_name in ['production_budget', 'domestic_gross', 'worldwide_gross']:
    df_budget = clean_dollar_values(df_budget, series_name)
df_budget['budget'] = np.log(df_budget.production_budget_int)
df_budget['domestic_revenues'] = np.log(df_budget.domestic_gross_int)
df_budget['world_revenues'] = np.log(df_budget.worldwide_gross_int)

In [ ]:
# Clean out outliers in budget and worldwide revenue series using
# calculate_whiskers function from Movie_Analysis_Functions Python file.
plt.boxplot([df_budget.budget, df_budget.world_revenues])
plt.set_xticklabels = [(1, 'Budget'), (2, 'Worldwide revenues')]
plt.show()
df_budget['budget_outliers'] = df_budget.loc[(df_budget.budget < calculate_whiskers(df_budget.budget)[0]) | \
                                             (df_budget.budget > calculate_whiskers(df_budget.budget)[1])].budget
df_budget = df_budget.loc[df_budget.budget_outliers.isna()]
df_budget['world_revenues_outliers'] = df_budget.loc[(df_budget.world_revenues < calculate_whiskers(df_budget.world_revenues)[0]) | \
                                                     (df_budget.world_revenues > calculate_whiskers(df_budget.world_revenues)[1])].world_revenues
df_budget = df_budget.loc[df_budget.world_revenues_outliers.isna()]

In [ ]:
# Show outliers for ROI
df_budget['roi'] = df_budget.world_revenues - df_budget.budget 
df_budget['roi_bottom'] = df_budget.loc[(df_budget.roi < calculate_whiskers(df_budget.roi)[0])].roi
df_budget['roi_top'] = df_budget.loc[(df_budget.roi > calculate_whiskers(df_budget.roi)[1])].roi
df_budget_roi_top = df_budget.loc[df_budget.roi_top.isna() == False]
df_budget_roi_bottom = df_budget.loc[df_budget.roi_bottom.isna() == False]
plt.scatter(df_budget.budget, df_budget.roi, color = 'grey', s = 3) 
plt.scatter(df_budget.budget, df_budget.roi_bottom, color = 'r') 
plt.scatter(df_budget.budget, df_budget.roi_top, color = 'g')
plt.xticks(np.arange(13, 20+(20-13)/4, (20-13)/4), (np.exp(np.arange(13, 20+(20-13)/4, (20-13)/4))*(10**(-6))).round(1), rotation = 90)
plt.xlabel('Budget (in millions)')
plt.yticks(np.arange(-6, 5+(5-(-6))/8, (5-(-6))/8), np.exp(np.arange(-6, 5+(5-(-6))/8, (5-(-6))/8)).round(2))
plt.ylabel('Return on investment\n(worldwide revenues/production budget)')
plt.title('Return on investment varies significantly')
plt.show()

In [ ]:
# Show ROI dispersion for <> $90 million movies
boxplot_data = [df_budget.loc[df_budget.production_budget_int <= 90*(10**6)].roi, \
                df_budget.loc[df_budget.production_budget_int > 90*(10**6)].roi]
plt.boxplot(boxplot_data)
plt.ylabel('Return on investment\n(Worldwide revenues/Production cost)')
plt.yticks(np.arange(-6, 5+(5-(-6))/8, (5-(-6))/8), \
           np.exp(np.arange(-6, 5+(5-(-6))/8, (5-(-6))/8)).round(2))
plt.xticks(np.arange(3), ('','Production budget\n<= $90 million', 'Production budget\n> $90 million'))
plt.title('Different business models')
plt.show()
lp_large = df_budget.loc[(df_budget.production_budget_int > 90*(10**6)) & (df_budget.roi < 0)] 
lp_small = df_budget.loc[(df_budget.production_budget_int <= 90*(10**6)) & (df_budget.roi < 0)]
# Show if there is a relationship between ROI and ratings
plt.scatter(np.log(df_budget.numvotes), df_budget.roi)
plt.scatter(np.log(df_budget.averagerating), df_budget.roi)
plt.scatter(np.log(df_budget.averagerating), np.log(df_budget.numvotes))

In [ ]:
# Prepare descriptive stats at the industry level
df_budget_annual_sum = df_budget.loc[(df_budget.release_year >=2010) & \
                                     (df_budget.release_year <= 2018)]. \
                                     groupby(['release_year']). \
                                     sum()
plt.fill_between(df_budget_annual_sum.index, \
                 (df_budget_annual_sum.production_budget_int)*(10**(-9)), \
                 (df_budget_annual_sum.worldwide_gross_int)*(10**(-9)), \
                 color='g')
plt.xlabel('Release year')
plt.ylabel('Production budget and worldwide revenues\n(in billions)')
plt.title('Increasing returns to movie production')
plt.show()
df_clean_annual_count = df_clean.loc[(df_clean.release_year >=2010) & \
                                     (df_clean.release_year <= 2018)]. \
                                     groupby(['release_year']). \
                                     title.count() 
plt.plot(df_clean.release_year, df_clean_annual_count)